---------------------------------------------
GLS Boxy: Skript pro čištění dat
---------------------------------------------
Tento skript provádí:
- Exploraci dat a zjištění rozsahu a kvality dat
- Vyčištění nepotřebných sloupců
- Filtrování pouze na boxy (ne pobočky)
- Odstranění třetích stran (např. AlzaBox)
- Vyčištění názvu boxů (name), měst (city)
- Porovnání city vs. city_name z RUIAN
- Rozpad sloupce address na street a house_no
- Nakonec fixace datových typů pro merge a SQL export

 1. Načtení pomocného datasetu s RUIAN kódem obce

In [24]:
import pandas as pd
data = pd.read_csv(r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\gls_pomocny_dataset_kod_obce.csv')

df = pd.DataFrame(data)

2. Explorace dat 
Parcel boxy: 2720 míst  
Parcel shopy: 1687 míst  
Sloupce s 1 hodnotou: 'ctrcode', 'iscodhandler', 'contact', 'phone', 'visible'  
Sloupce: parcelBox a isparcellocker	jsou totožné, Parcelbox/ParcelShop  

In [25]:
df.info

<bound method DataFrame.info of                                     address             pclshopid  \
0                     S. K. Neumanna 590/22  CZ58601-PARCELLOCK01   
1                          Myslotínská 2449         39301-ELPESRO   
2     Národní obrany 824/35 - Pasáž Riviera         16900-GLSCZPS   
3                          Kolbenova 650/22       19000-FAMIKLSRO   
4                           Nerudova 1357/5      58901-RICHTEROVA   
...                                     ...                   ...   
4402                        Hostovského 270      CZ54931-PSHOP001   
4403                   Cukrovarská 1141/47b      CZ19600-PSHOP002   
4404                     Jeronýmova 569/22a      CZ46007-PSHOP006   
4405                       Počernická 732/5      CZ11000-PSHOP006   
4406                                Lužická    CZ47125-PLOCKER002   

               _leaflet_id ctrcode                   city     geolat  \
0     CZ58601-PARCELLOCK01      CZ                Jihlava  49.40480

In [26]:
df.head(2)

,address,pclshopid,_leaflet_id,ctrcode,city,geolat,geolng,iscodhandler,isparcellocker,contact,...,wheelchairAccess,zipcode,openSaturday,openSunday,imageUrl,holiday,city_code,city_name,search_term,search_type
0,S. K. Neumanna 590/22,CZ58601-PARCELLOCK01,CZ58601-PARCELLOCK01,CZ,Jihlava,49.4048,15.557,1,1,NaN,...,NaN,58601,1,1,https://prodpdmpublicstore.blob.core.windows.n...,NaN,586846.0,Jihlava,S. K. Neumanna 590/22 Jihlava 58601,full
1,Myslotínská 2449,39301-ELPESRO,39301-ELPESRO,CZ,Pelhřimov,49.4139,15.220,1,0,NaN,...,NaN,39301,1,0,https://prodpdmpublicstore.blob.core.windows.n...,NaN,547492.0,Pelhřimov,Myslotínská 2449 Pelhřimov 39301,full


In [27]:
df.shape

(4407, 30)

Výskyt a počet Nan - pouze COL 'holiday'

In [28]:
df.isnull().sum()

address                0
pclshopid              0
_leaflet_id            0
ctrcode                0
city                   0
geolat                 0
geolng                 0
iscodhandler           0
isparcellocker         0
contact             4407
phone               4407
name                   0
guid                   0
visible                0
utilization            0
capacitystate          0
parcelBox              0
parcelShop             0
payCard                0
paybybankcard          0
wheelchairAccess    3524
zipcode                0
openSaturday           0
openSunday             0
imageUrl             772
holiday             4161
city_code             40
city_name             40
search_term           40
search_type           40
dtype: int64

přehled kategorií/textových sloupců:

In [29]:
df.select_dtypes(include='object').nunique()

address        4363
pclshopid      4407
_leaflet_id    4407
ctrcode           1
city           1367
name           3310
guid           4389
imageUrl       3627
holiday          51
city_name      1016
search_term    4242
search_type       4
dtype: int64

sloupce, seřazené dle hodnot

In [30]:
df.nunique().sort_values()

phone                  0
contact                0
ctrcode                1
iscodhandler           1
visible                1
wheelchairAccess       1
parcelBox              2
isparcellocker         2
parcelShop             2
openSunday             2
openSaturday           2
paybybankcard          2
payCard                2
capacitystate          3
search_type            4
utilization           36
holiday               51
city_name           1016
city_code           1033
zipcode             1138
city                1367
name                3310
imageUrl            3627
search_term         4242
geolat              4263
geolng              4319
address             4363
guid                4389
_leaflet_id         4407
pclshopid           4407
dtype: int64

Odstranit sloupce s 1 unikátní hodnotou:

In [31]:
df.columns[df.nunique() == 1]

Index(['ctrcode', 'iscodhandler', 'visible', 'wheelchairAccess'], dtype='object')

In [32]:
df = df.drop(columns = df.columns[df.nunique()==1])

In [33]:
df['isparcellocker'].value_counts()


isparcellocker
1    2720
0    1687
Name: count, dtype: int64

In [34]:
df.columns[df.nunique() != 1]

Index(['address', 'pclshopid', '_leaflet_id', 'city', 'geolat', 'geolng',
       'isparcellocker', 'contact', 'phone', 'name', 'guid', 'utilization',
       'capacitystate', 'parcelBox', 'parcelShop', 'payCard', 'paybybankcard',
       'zipcode', 'openSaturday', 'openSunday', 'imageUrl', 'holiday',
       'city_code', 'city_name', 'search_term', 'search_type'],
      dtype='object')

 3. Vyčistění na úrovni boxů  
    ponechat pouze parcel locker (ne pobočky)  
    Sloupec  ISPARCELLOCKER 1 = BOX,  0 = výdejní místo nebo provozovna, depo  

In [35]:
df = df[df['isparcellocker'] == 1]

3. Zachovat a seřadit pouze klíčové sloupce pro další čištění

In [36]:
df = df[['name', 'city_code', 'city_name', 'city', 'zipcode', 'address', 'geolat', 'geolng']]
df.head(3)

,name,city_code,city_name,city,zipcode,address,geolat,geolng
0,GLS Parcel Box,586846.0,Jihlava,Jihlava,58601,S. K. Neumanna 590/22,49.4048,15.5570
743,GLS Parcel Box,597007.0,Velké Meziříčí,Velké Meziříčí,59401,Sokolovská 147/25,49.3517,16.0196
747,GLS Parcel Box,545058.0,Valašské Meziříčí,Valašské Meziříčí,75701,Králova 1256,49.4693,17.9711


4. Odstranění AlzaBoxů a AlzaWall  
   DF obsahuje boxy od 3. stran AlzaBox a AlzaWall (to je v Alzaprodejně)  
   po odstranění zůstává 973 záznamů  

In [37]:
alza_boxwall = df[df['name'].str.contains(r'AlzaBox|AlzaWall', na=False)]
alza_boxwall.shape

(1747, 8)

In [38]:
df = df[~df['name'].str.contains(r'AlzaBox|AlzaWall', na=False)]
df.shape

(973, 8)

5. Vyčistění NAME (odstranit závorky)  

In [39]:
name_se_zavorkou= df[df['name'].str.contains(r'\(.*\)$', na=False)]
name_se_zavorkou

,name,city_code,city_name,city,zipcode,address,geolat,geolng
752,GLS Parcel Box (NC Haná),500496.0,Olomouc,Olomouc,77900,Kafkova 1223/8 (OC Haná),49.575400,17.228200
763,GLS Parcel Box (Lékárna u sv. Anny),582786.0,Brno,Brno,60200,Mendlovo náměstí,49.190446,16.596150
782,GLS Parcel Box (NC Fénix),554782.0,Praha,Praha,19000,Freyova 945,50.109900,14.501600
800,GLS Parcel Box (Sportovní hala Lužiny),554782.0,Praha,Praha,15500,Bellušova 1877/68,50.041100,14.325200
822,GLS Parcel Box (OC Atrium),569810.0,Hradec Králové,Hradec Králové,50002,Dukelská třída 1713/7,50.213200,15.818800
837,GLS Parcel Box (Homepark Zličín),554782.0,Praha,Praha,15500,Skandinávská 144/25,50.050118,14.294846
841,GLS Parcel Box (NORMA),539333.0,Jílové u Prahy,Jílové u Prahy,25401,Rudných dolů 502,49.896700,14.492400
843,GLS Parcel Box (Menza ČVUT),554782.0,Praha,Praha,16000,Kolejní 2571,50.105594,14.389182
847,GLS Parcel Box (OMV),554821.0,Ostrava,Ostrava,70200,Mariánskohorská 3121/55,49.840600,18.260400
872,GLS Parcel Box (OC Futurum),582786.0,Brno,Brno,61900,Vídeňská 132/100,49.155800,16.597600


In [40]:
df.loc[:, 'name'] = df['name'].str.replace(r'\s*\(.*\)$', '', regex=True)
name_se_zavorkou= df[df['name'].str.contains(r'\(.*\)$', na=False)]
name_se_zavorkou

,name,city_code,city_name,city,zipcode,address,geolat,geolng


6. Vyčistění city_code od .0 (pokud se objeví)  

In [41]:
df.loc[:, 'city_code'] = df['city_code'].astype(str).str.replace(r'\.0$', '', regex=True)

C:\Users\Sabina\AppData\Local\Temp\ipykernel_25888\2544267403.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['586846' '597007' '545058' '554782' '511382' '585068' '592731' '500496'
 '555088' '582786' '598003' '554821' '500496' '544256' '582786' '545881'
 '531057' '544256' '554782' '554791' '555771' '553425' '545881' '554782'
 '582786' '582786' '554782' '554782' '582786' '585068' '554782' '573868'
 '554782' '555134' '569810' '582786' '554782' '554782' '554782' '554782'
 '539333' '555134' '554782' '554782' '569810' '554782' '554821' '582786'
 '574121' '554791' '573868' '554782' '554782' '554782' '582786' '544256'
 '554782' '561380' '532053' '554791' '554782' '555088' '532819' '554782'
 '582786' '554782' '598003' '554782' '554782' '598003' '554481' '554782'
 '567027' '554791' '582786' '554782' '567027' '554821' '554961' '554782'
 '554791' '569810' '500496' '544256' '554782' '554782' '554821' '552046'
 '554804' '

7. Vyčistění CITY  
   nejprve sjednocení "title"  
   odstranění závorek s chybou (např. "X)")

In [42]:
df['city'] = df['city'].str.strip().str.title()

In [43]:
city_Xrozbite= df[df['city'].str.contains(r'\s*X\).*', na=False)]
city_Xrozbite

,name,city_code,city_name,city,zipcode,address,geolat,geolng
2058,GLS Parcel Box,582786,Brno,Brno-Slatina X),62700,Tuřanka 1260/1,49.176424,16.688113
2265,GLS Parcel Box,554821,Ostrava,Ostrava-Zabreh X),70030,Výškovická 2636/116,49.790598,18.227767
2356,GLS Parcel Box,582786,Brno,Brno-Bohunice X) (Brno-Me,63900,Vídeňská 815/89a,49.175910,16.597449
2357,GLS Parcel Box,582786,Brno,Brno-Kohoutovice X) (Brno,62300,Libušina tř. 580/4,49.197138,16.533717
2415,GLS Parcel Box,582786,Brno,Brno-Lisen X),62800,Sedláčkova 2557/4,49.202316,16.668318
2620,GLS Parcel Box,554821,Ostrava,Ostrava-Poruba X),70800,Pavlouskova 4457/24,49.838772,18.163377
2621,GLS Parcel Box,554821,Ostrava,Ostrava-Poruba X),70800,Oty Synka 1876/8,49.845491,18.160582
3913,GLS Parcel Box,nan,NaN,Brno-Cernovice X) (Brno-M,61800,Blatouchová 1074/2b,49.185489,16.632651


In [44]:
df['city'] = df['city'].str.replace(r'\s*X\).*', '', regex=True)

Porovnání city(gls dataset) vs. city_name(RUIAN) (shoda s RUIANEM)  
   ponechávám hodnoty z RUIAN ve sloupci city_name jako důvěrohodnější,
   a sloupec city (z původních dat) může být odstraněn (bude provedeno až 
   na konci při výběru a přejmenování sloupců). Sloupec proto není vyčištěn
   pořádně.

In [45]:
df['shoda_city'] = df['city'] == df['city_name']
# Výběr jen těch řádků, kde se město neshoduje
neshody = df[df['shoda_city'] == False]

In [46]:
neshody[['city', 'city_name', 'shoda_city']].head()

,city,city_name,shoda_city
841,Jílové U Prahy,Jílové u Prahy,False
1516,Frýdek Místek,Frýdek-Místek,False
1519,Frýdek Místek,Frýdek-Místek,False
1548,Ústí Nad Labem,Ústí nad Labem,False
1855,Chomutov,Otvice,False


In [47]:
city_se_zavorkou = df[df['city'].str.contains(r'\(.*\)$', na=False)]

city_se_zavorkou

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city
1985,GLS Parcel Box,552046,Tábor,Tabor (Tabor),39005,Helsinská 2760/18,49.385556,14.688058,False
2070,GLS Parcel Box,562971,Chomutov,Chomutov (Chomutov),43004,Pod Břízami 5251,50.491305,13.438580,False
2118,GLS Parcel Box,567027,Most,Most (Most),43401,Rudolická 1706/4,50.510646,13.651240,False
2167,GLS Parcel Box,532053,Kladno,Kladno (Kladn),27201,Americká 3321,50.128726,14.116330,False
2169,GLS Parcel Box,532053,Kladno,Kladno (Kladn),27201,Náměstí Sítná 3127,50.135393,14.102904,False
2244,GLS Parcel Box,579025,Trutnov,Vnitrni Mesto (Trutnov),54101,Na Struze 30,50.562256,15.918045,False
2246,GLS Parcel Box,581283,Blansko,Blansko (Blansko),67801,Poříčí 2610,49.368950,16.640350,False
2249,GLS Parcel Box,534676,Mělník,Melnik (Melnik),27601,Vodárenská 219,50.352256,14.480146,False
2250,GLS Parcel Box,532053,Kladno,Kladno (Kladn),27201,Hřebečská 2732,50.132740,14.137471,False
2306,GLS Parcel Box,585599,Otrokovice,Otrokovice (Zlin),76502,Příčná 1580,49.203706,17.538262,False


In [48]:
mask = df['city'].str.contains(r'\(.*\)$', na=False)
df.loc[mask, 'city'] = df.loc[mask, 'city_name']

In [49]:
df[df['city'].str.contains(r'\(.*\)$', na=False)]

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city


8. Rozpad sloupce address na address a house_no

In [50]:
df.loc[:,'house_no'] = df['address'].str.extract(r'(\d[\dA-Za-z/,\- ]*(/[IVXLCDM]+)?)$')[0]
df.loc[:,'address'] = df['address'].str.replace(r'\s\d[\dA-Za-z/,\- ]*(/[IVXLCDM]+)?$', '', regex = True).str.strip()

In [51]:
adresy_bez_cisla = df[~df['address'].str.contains(r'\s\d[\dA-Za-z/,\- ]*(?:/[IVXLCDM]+)?$', na=False)]
adresy_bez_cisla

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,house_no
0,GLS Parcel Box,586846,Jihlava,Jihlava,58601,S. K. Neumanna,49.404800,15.557000,True,590/22
743,GLS Parcel Box,597007,Velké Meziříčí,Velké Meziříčí,59401,Sokolovská,49.351700,16.019600,True,147/25
747,GLS Parcel Box,545058,Valašské Meziříčí,Valašské Meziříčí,75701,Králova,49.469300,17.971100,True,1256
748,GLS Parcel Box,554782,Praha,Praha,15600,Pod Třešňovkou,49.970800,14.383200,True,234
749,GLS Parcel Box,511382,Přerov,Přerov,75002,Svisle,49.450300,17.459400,True,1100/9
...,...,...,...,...,...,...,...,...,...,...
4271,GLS Parcel Box,585068,Zlín,Zlín,76001,Ladislava Váchy,49.217261,17.633535,True,517
4272,GLS Parcel Box,582786,Brno,Brno,62700,Langrova,49.179369,16.679533,True,1227/1d
4301,GLS Parcel Box,596230,Nové Město na Moravě,Nové Město Na Moravě,59231,Komenského náměstí,49.563083,16.075644,False,999
4309,GLS Parcel Box,583782,Rosice,Rosice U Brna,66501,Zastávecká,49.184816,16.368091,False,1267


In [52]:
adresy_s_pomlckou = df[df['address'].str.contains(r'-', na=False)]
adresy_s_pomlckou

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,house_no
777,GLS Parcel Box,582786,Brno,Brno,62500,Netroufalky 770/14-16 (OC Campus Square),49.177700,16.566800,True,NaN
1909,GLS Parcel Box,536385,Jeseník,Jeseník,79001,Autobusové n.-Smetanova ul.,50.227500,17.208300,True,NaN
2358,GLS Parcel Box,554782,Praha,Praha,17000,Veletržní 1623/24 (-1P Garáže),50.101736,14.430196,True,NaN
2864,GLS Parcel Box,553271,Veselí nad Lužnicí,Veselí Nad Lužnicí,39181,Tř. Československé Armády 609 - u myček,49.197283,14.716687,False,NaN
2870,GLS Parcel Box,588458,Holešov,Holešov,76901,Obchodní galerie - ulice Tovární,49.327873,17.573918,True,511
2960,GLS Parcel Box,578347,Litomyšl,Litomyšl,57001,Sokolovská 113 - za automyčkou,49.880838,16.292915,True,NaN
3096,GLS Parcel Box,561380,Česká Lípa,Česká Lípa,47001,Na Blatech - automyčka DIBO,50.687907,14.519294,True,NaN
3106,GLS Parcel Box,573868,Náchod,Náchod,54701,Mlýnská - parkoviště,50.414193,16.162277,True,NaN
3133,GLS Parcel Box,576069,Rychnov nad Kněžnou,Rychnov Nad Kněžnou,51601,Zborovská 17 - vedle katastrálního úřadu,50.163417,16.278833,False,NaN
3207,GLS Parcel Box,554782,Praha,Praha,14700,Ke Krči - zastávka autobusu Branické náměstí,50.034667,14.414569,True,NaN


Odstranit text za č.p
Je jednodušší odstranit specifické údaje, než uvažovat nad regexem.

In [53]:
specificke_adresy = df[df['address'].str.contains(' - parkoviště| - u myček| - Parkoviště| - za automyčkou| - vedle katastrálního úřadu| - vjezd z ulice Jizbická', na=False)]

df.loc[:,'address'] = df['address'].str.replace(' - parkoviště| - u myček| - Parkoviště| - za automyčkou| - vedle katastrálního úřadu| - vjezd z ulice Jizbická', '', regex = True)


In [54]:
specificke_adresy = df[df['address'].str.contains(' - parkoviště| - u myček| - Parkoviště| - za automyčkou| - vedle katastrálního úřadu| - vjezd z ulice Jizbická', na=False)]
specificke_adresy

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,house_no


Kontrola, kde chybí house_no. Pokud zůstalo v address doplním v dalším kroku
Odstranit z pole address vše v závorce zřejmě není správné a znehodnotí adresu.
Vizualizace se bude dít na úrovni obce, pokud by se ukázalo, že je sloupec address
(ulice) potřebné, potom dočístím následně.

In [55]:
chybi_house_no = df[df['house_no'].isna()]
chybi_house_no

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,house_no
752,GLS Parcel Box,500496,Olomouc,Olomouc,77900,Kafkova 1223/8 (OC Haná),49.575400,17.228200,True,NaN
761,GLS Parcel Box,555088,Havířov,Havířov,73601,Dělnická (OC Elan),49.780800,18.431800,True,NaN
763,GLS Parcel Box,582786,Brno,Brno,60200,Mendlovo náměstí,49.190446,16.596150,True,NaN
766,GLS Parcel Box,554821,Ostrava,Ostrava,72527,Spojovací 396 (NC Géčko),49.849000,18.145800,True,NaN
776,GLS Parcel Box,544256,České Budějovice,České Budějovice,37001,Nádražní 1759 (OC Mercury),48.972900,14.487400,True,NaN
777,GLS Parcel Box,582786,Brno,Brno,62500,Netroufalky 770/14-16 (OC Campus Square),49.177700,16.566800,True,NaN
796,GLS Parcel Box,554782,Praha,Praha,14000,Na Hřebenech II,50.053585,14.430863,True,NaN
838,GLS Parcel Box,554782,Praha,Praha,16900,Chaloupeckého 1914/9 (hřiště proti bloku 4),50.080300,14.392800,True,NaN
1531,GLS Parcel Box,554821,Ostrava,Ostrava,70030,Dr. Lukášové křižovatka Holasova,49.787700,18.256500,True,NaN
1909,GLS Parcel Box,536385,Jeseník,Jeseník,79001,Autobusové n.-Smetanova ul.,50.227500,17.208300,True,NaN


In [56]:
doplnit_house_no = {
    752: '1223/8', 
    766: '396',
    776: '1759',
    777: '770/14-16',
    838: '1914/9',
    2358: '1623/24',
    3069: '1531/3',
    3771: '139',    
    2864: '609',    
    2917: '269',
    2960: '113',
    3133: '17',
    3466: '23',
    3532: '2292/14',
    3995: '239'
    # ...
}

# Přiřazení hodnot do sloupce house_no
for idx, cislo in doplnit_house_no.items():
    df.loc[idx, 'house_no'] = cislo

# Volitelná kontrola
df.loc[doplnit_house_no.keys(), ['address', 'house_no']]

,address,house_no
752,Kafkova 1223/8 (OC Haná),1223/8
766,Spojovací 396 (NC Géčko),396
776,Nádražní 1759 (OC Mercury),1759
777,Netroufalky 770/14-16 (OC Campus Square),770/14-16
838,Chaloupeckého 1914/9 (hřiště proti bloku 4),1914/9
2358,Veletržní 1623/24 (-1P Garáže),1623/24
3069,Duhová 1531/3 (zadní strana parkoviště),1531/3
3771,náměstí Míru 139 (u radnice),139
2864,Tř. Československé Armády 609,609
2917,"Aléská 269, Pražské předměstí",269


9. Doplnit chybějící údaje city_code    
   Vytvářím ID, kontrolní údaje mám v kontrolní csv souboru s ID, se kterým chci   
   párovat. Tyto údaje nebyly nalezeny programem assign_address... (autor Kačka  
   Jašková), doplnila jsem ručně, nyní jen nahrávám do souboru.

In [57]:
chybi_city_code = df[df['city_code'].isna()]
chybi_city_code

,name,city_code,city_name,city,zipcode,address,geolat,geolng,shoda_city,house_no


In [58]:
df['id'] = range(1, len(df) + 1)

In [59]:
# Slovník s doplňovanými hodnotami podle sloupce 'id': city_code a správné city
doplnit = {
    863: (582786, "Brno-Černovice"),
    195: (584291, "Břeclav"),
    544: (544256, "České Budějovice"),
    206: (598003, "Frýdek-Místek"),
    335: (580511, "Lanškroun"),
    366: (580511, "Lanškroun"),
    534: (548511, "Pacov"),
    836: (555134, "Pardubice"),
    214: (545058, "Valašské Meziříčí")
}

# Doplnění hodnot do sloupců 'city_code' a 'city' podle 'id'
for idx, (kod, mesto) in doplnit.items():
    df.loc[df['id'] == idx, 'city_code'] = kod
    df.loc[df['id'] == idx, 'city_name'] = mesto

# Kontrola záznamů po úpravě
df[df['id'].isin(doplnit.keys())][['id', 'name', 'city_code', 'city_name']]

,id,name,city_code,city_name
2100,195,GLS Parcel Box,584291,Břeclav
2114,206,GLS Parcel Box,598003,Frýdek-Místek
2122,214,GLS Parcel Box,545058,Valašské Meziříčí
2653,335,GLS Parcel Box,580511,Lanškroun
2773,366,GLS Parcel Box,580511,Lanškroun
3192,534,GLS Parcel Box,548511,Pacov
3202,544,GLS Parcel Box,544256,České Budějovice
3866,836,GLS Parcel Box,555134,Pardubice
3913,863,GLS Parcel Box,582786,Brno-Černovice


10. Finální úprava před nahráním souboru do databáze SQL (nebo mergem s dalšími)

Odstranit sloupec city (vysvětleno v bodu 8)

In [60]:
df = df.drop(columns=['city'])

Pojmenovat, vybrat a seřadit sloupce  

In [61]:
df = df.rename(columns={
    'address': 'street',
    'zipcode': 'zip_code',
    'house_no': 'house_NO',
    'city_name': 'city',
    'geolat': 'latitude', 
    'geolng': 'longtitude'
  })

df = df[['id', 'name', 'city_code', 'city', 'zip_code', 'street', 'house_NO',  'latitude', 'longtitude']]
df.head(3)

,id,name,city_code,city,zip_code,street,house_NO,latitude,longtitude
0,1,GLS Parcel Box,586846,Jihlava,58601,S. K. Neumanna,590/22,49.4048,15.5570
743,2,GLS Parcel Box,597007,Velké Meziříčí,59401,Sokolovská,147/25,49.3517,16.0196
747,3,GLS Parcel Box,545058,Valašské Meziříčí,75701,Králova,1256,49.4693,17.9711


Fix datových typů pro merge a SQL export

In [62]:
df['house_NO'] = df['house_NO'].astype(str)
df['zip_code'] = df['zip_code'].astype(str)
df['city_code'] = df['city_code'].astype(str)
df['street'] = df['street'].astype(str)
df['latitude'] = df['latitude'].astype(float)
df['longtitude'] = df['longtitude'].astype(float)

In [63]:
df.columns

Index(['id', 'name', 'city_code', 'city', 'zip_code', 'street', 'house_NO',
       'latitude', 'longtitude'],
      dtype='object')

In [64]:
df[['id', 'name', 'city_code', 'city', 'zip_code', 'street', 'house_NO',
       'latitude', 'longtitude']].dtypes

id              int64
name           object
city_code      object
city           object
zip_code       object
street         object
house_NO       object
latitude      float64
longtitude    float64
dtype: object

In [65]:
# df.to_csv(r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\gls_pomocny_dataset.csv', index=False, encoding='utf-8')

Finální dataset

In [66]:
df.to_csv(r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\ss_gls_clean_dataset.csv', index=False, encoding='utf-8')

In [67]:
# import json

# df.to_json('ss_clean_data_gls.json', orient='records', force_ascii=False, indent=4)